# Approximators

> Models that approximate a function

In [ ]:
#| default_exp approximators

In [ ]:
import logging
logging_level = logging.DEBUG

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from abc import ABC, abstractmethod
from typing import Union
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import time

In [ ]:
#| export
class LinearModel(nn.Module):
    """Linear regression model"""

    def __init__(self, 
            input_size: int, # number of features
            output_size: int, # number of outputs/actions
            relu_output: bool = False): # whether to apply ReLU activation to the output
        super().__init__()
        self.l1=nn.Linear(input_size, output_size)
        if relu_output:
            self.final_activation = nn.ReLU()
        else:
            self.final_activation = nn.Identity()
            
    def forward(self,x):
        out=self.l1(x)
        out=self.final_activation(out)
        return out

In [ ]:
#| export
class MLP(nn.Module):

    """ Multilayer perceptron model"""

    def __init__(self,
                    input_size: int, # number of features
                    output_size: int, # number of outputs/actions
                    hidden_layers: list, # list of number of neurons in each hidden layer
                    drop_prob: float = 0.0, # dropout probability
                    batch_norm: bool = False, # whether to apply batch normalization
                    relu_output: bool = False): # whether to apply ReLU activation to the output
        super().__init__()

        # List of layers
        layers = []

        last_size = input_size
        for num_neurons in hidden_layers:
            layers.append(nn.Linear(last_size, num_neurons))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(p=drop_prob))
            if batch_norm:
                layers.append(nn.BatchNorm1d(num_neurons))
            last_size = num_neurons

        # Output layer
        layers.append(nn.Linear(last_size, output_size))
        if relu_output:
            layers.append(nn.ReLU()) # output is non-negative
        else:
            layers.append(nn.Identity())

        # Combine layers
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()